In [1]:
# the packages
import pandas as pd
import numpy as np
import networkx as nx

# import my own helper functions
from read import read_sims_result
from clean import cleanup_0IR_exp
from clean import cleanup_network

# pearson correlation coeffcient
from scipy.stats.stats import pearsonr

# deep copy
from copy import deepcopy

# Page Rank
from networkx.algorithms.link_analysis.pagerank_alg import pagerank, pagerank_numpy, pagerank_scipy

# my own customized Page Rank
from my_page_rank import *

# distance
from networkx.algorithms.shortest_paths.generic import shortest_path_length

# logistic regression
from sklearn.linear_model import LogisticRegression

# cross validation
from sklearn.model_selection import KFold

# confusion matrix
from sklearn.metrics import confusion_matrix

# plot
import matplotlib.pyplot as plt

In [2]:
# independent variables
independent = ["deposits", "cash", "assets", "credit available", "wealth", "leverage", 
         "dummy-0-leverage",
         "wealth-lag", "deposits-lag", "cash-lag", "assets-lag", "leverage-lag", 
         "credit-available-lag", "credit-issued-lag", "dummy-0-leverage-lag",
         "over-leverage-frequency"]

In [3]:
# ###########################
# Read OIR results, and fit the model
# ###########################
df0 = read_sims_result("/Users/xcheng/Documents/Oberlin/Summer2/DataAnalysis/data/0622/0IR300s", 32)
df0c = cleanup_0IR_exp(df0, numNode=32, numPeriod=15, numSim=100, balanced=True)

X = df0c[independent]
y = df0c["default-next"]

final = LogisticRegression(penalty="l1", C=0.007)
final.fit(X,y)

LogisticRegression(C=0.007, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [4]:
# make sure no defaults in 0IR
# sum(df0[df0["defaults due to interest"]
#     +df0["defaults due to negative wealth"]
#     +df0["defaults due to deposit shock"] == 0].loc[:,"dot0":"dot30"].values)

In [5]:
# ###########################
# Read & process positive IR results
# ###########################
df_1 = read_sims_result("/Users/xcheng/Documents/Oberlin/Summer2/DataAnalysis/data/0625/1IR", 32)
mx_1n = cleanup_network(df_1, numNode=32, numPeriod=15, numSim=50)
df_1c = cleanup_0IR_exp(df_1, numNode=32, numPeriod=15, numSim=50)

In [6]:
def create_edge_weight(N, dff, mid):
    """
    Calculate weight for edges
    Each debt is divided by lenders' wealth (w/o haircut)，
    The result number r is scaled to [0, 1) using g(r)=r/(mid+r)
    
    Parameters
    ----------
    N: 4D numpy array [n_simulations, n_periods, n_borrowers, n_lenders]
        debt adjacency matrices 
    df: Pandas dataframe (no cleanup)
        where we get banks' wealth (w/o haircut)
    mid: int
        the debt-to-wealth ratio resulting in 50% probability of spreading default
    
    Returns
    ----------
    WN: 4D numpy array [n_simulations, n_periods, n_borrowers, n_lenders]
        new weighted debt adjacency matrices
    """
    
    WN = np.copy(N)
    simNum, periodNum, bankNum, _= N.shape
    dff["book_wealth"] = (dff["assets"] + dff["cash"] + dff["debt owed"] 
                         - dff["debt to pay"] - dff["deposits"])
    
    for s in range(simNum):
        for p in range(1,periodNum+1):
            for lender in range(bankNum):
                w = dff[np.array(dff["sim#"]==s) &
                        np.array(dff["period"]==p) & 
                        np.array(dff["bankID"]==lender)
                       ]["book_wealth"].values[0]
                
                # helper function
                def f(a):
                    if a > 0: # there is debt
                        if w > 0: # positive wealth
                            t = a/w
                            return t/(t+mid)
                        else: # 0 or negative wealth
                            return 100/(100+mid)
                    else: # no debt or weird data 
                        return 0
                
                WN[s, p-1, :, lender] = [f(k) for k in WN[s, p-1, :, lender]]
                    
    return WN

In [7]:
def create_node_weight(N, dff, model, variables):
    """
    Calculate weight for nodes
    Each debt is multiplied by lenders' predicted default probability
    
    Parameters
    ----------
    N: 4D numpy array [n_simulations, n_periods, n_borrowers, n_lenders]
        debt adjacency matrices 
    dff: Pandas dataframe (yes cleanup)
        where we get bank's balance sheet info
    model: model for default probability
        scikit learn LogisticRegression
    variables: a list of strings
        independent variables for the model
    
    Returns
    ----------
    WN: 3D numpy array [n_simulations, n_periods, n_banks]
        array of predicted probability of default
    """
    
    simNum, periodNum, bankNum, _= N.shape
    WN = np.empty((simNum, periodNum, bankNum))
    WN.fill(-1)
    
    for s in range(simNum):
        for p in range(2,periodNum):
            for b in range(bankNum):
                X = dff[np.array(dff["sim#"]==s) &
                        np.array(dff["period"]==p) & 
                        np.array(dff["bankID"]==b)
                       ][variables].values
                if X.any():
                    predicted_default_probability = model.predict_proba(X)[0][1]
                    WN[s, p-1, b] = predicted_default_probability
                    
    return WN

In [8]:
edge_weights_all = create_edge_weight(mx_1n, df_1, 0.6)
node_weights_all = create_node_weight(mx_1n, df_1c, final, independent)

Empirically, I discover that it does not matter 0.6 is. I experimented with serveral numbers between 0 and 1.

In [9]:
# visualize the debt adjacency matrix in a 2d graph
# plt.figure(figsize=(10,10))
# plt.imshow(edge_weights_all[0,1], interpolation='nearest')
# plt.show()

In [10]:
def customized_random_walk_exp(N, solvent, random_walk, iterations=10):
    """
    NEW NEW NEW modified random walk algorithm
    
    Parameters
    ----------
    N: 4D numpy array [n_simulations, n_periods, n_borrowers, n_lenders]
        debt adjacency matrices 
    solvent: 3D numpy array [n_simulations, n_periods, n_borrowers, n_lenders] 
        predicted default probability OR -1
    iterations: int
        number of iterations
    
    Returns
    ----------
    result: 1D numpy array
        index for the bank's default probability   
    """
    n_s, n_p, n_b, _ = N.shape
    s_s, s_p, s_b = solvent.shape
    big_result = []
    
    if n_s != s_s or n_p != s_p or n_b != s_b:
        raise ValueError('Two arrays have incompatible sizes.')
        
    for i, j in np.ndindex((n_s,n_p)):
        big_result.extend(random_walk(N[i,j],
                                      solvent[i,j],
                                      iterations=iterations
                                     ).values())
        
    return np.array(big_result)

In [33]:
overall_prediction = customized_random_walk_exp(edge_weights_all, node_weights_all, random_walk_single_1b, 500)

In [19]:
len(overall_prediction)

18240

In [20]:
# ###########################
# pearson correlation coeffcient is not very big 
# it gets slightly bigger with lots of iterations
# ###########################
pearsonr(overall_prediction, df_1c["default-next"])

(0.30186645378375887, 0.0)

In [21]:
df_1c.groupby("default-next").count()

,sim#,period,bankID,theta (risk aversion),wealth,deposits,cash,assets,leverage,credit available,...,cash-lag,assets-lag,leverage-lag,credit-available-lag,credit-issued-lag,dummy-0-leverage-lag,over-leverage-frequency,default-next-wealth,default-next-deposit,default-next-interest
default-next,,,,,,,,,,,,,,,,,,,,,
0,18079,18079,18079,18079,18079,18079,18079,18079,18079,18079,...,18079,18079,18079,18079,18079,18079,18079,18079,18079,18079
1,161,161,161,161,161,161,161,161,161,161,...,161,161,161,161,161,161,161,161,161,161


In [22]:
def customized_k_fold(X, y, fold=12):
    """
    customized K-fold cross validation 
    print certain summary statistics
    """
    # k-fold
    kf = KFold(n_splits=fold, shuffle=True)

    # initialization
    accuracy = 0
    conf = np.array([[0, 0], [0, 0]])

    def g(A, arr):
        if A.shape[1] == 1: return A[arr]
        else: return A.loc[arr]
    
    for train_index, test_index in kf.split(X):
#         print(train_index, test_index)
        
        model = LogisticRegression()
        model.fit(g(X,train_index), y.iloc[train_index])
        print(model.coef_, model.intercept_)

        accuracy += model.score(g(X,test_index), y.iloc[test_index])
        conf += confusion_matrix(y.iloc[test_index], model.predict(g(X,test_index)))

    tn, fp, fn, tp = conf.ravel()
    print("{}\n accuracy:{:24}\n precision:{:24}\n recall:{:24}\n".format(
            conf, 
            accuracy/fold, 
            tp/(tp+fp),
            tp/(tp+fn)))

In [23]:
def customized_k_fold_balanced(X, y, fold=12):
    """
    customized K-fold cross validation 
    print certain summary statistics
    """
    all_obs = np.array(range(len(y)))
    default_obs = all_obs[y==1]
    nondefault_obs = np.random.choice(all_obs[y==0],
                                     size=len(default_obs),
                                     replace=False)
    all_index = np.append(default_obs, nondefault_obs)

    if X.shape[1]== 1:
        customized_k_fold(X[all_index], y[all_index], fold=fold)       
    else:
        customized_k_fold(X.loc[all_index].reset_index(drop=True), 
                          y[all_index], 
                          fold=fold)

In [35]:
# 1111111111111111111111111
# K-fold cross validation 
# use approx. default probability (network info) to predict default
# ###########################
customized_k_fold_balanced(overall_prediction.reshape(-1, 1), df_1c["default-next"])

[[0.01361869]] [-3.90726054]
[[0.01397946]] [-3.94548722]
[[0.01376178]] [-3.86990437]
[[0.0143503]] [-4.03719719]
[[0.01384686]] [-3.92518404]
[[0.01385552]] [-3.91118102]
[[0.01368166]] [-3.90414188]
[[0.01443893]] [-3.98620435]
[[0.01418632]] [-3.99821625]
[[0.01390689]] [-3.98062103]
[[0.01370843]] [-3.86930933]
[[0.01473468]] [-4.06248995]
[[152   9]
 [  3 158]]
 accuracy:      0.9627255460588794
 precision:      0.9461077844311377
 recall:      0.9813664596273292



In [43]:
# customized_k_fold_balanced(df_1c["assets"].reshape(-1, 1), df_1c["default-next"])

In [44]:
# customized_k_fold_balanced(node_weights_all.reshape(-1, 1), df_1c["default-next"])

Now we want train on one balanced dataset, and then test on a different unbalanced dataset.

We still the low precision, high recall.

In [36]:
def train_balanced(X, y):
    """
    Balanced  the  data, and train a model
    
    Parameters
    ----------
    X: array-like
        training data: independent variables
    y: array-like
        training data: dependent variables
    
    Returns
    ----------
    model: Logisitc regression. 
    """
    # balance the data
    all_obs = np.array(range(len(y)))
    default_obs = all_obs[y==1]
    nondefault_obs = np.random.choice(all_obs[y==0],
                                     size=len(default_obs),
                                     replace=False)
    all_index = np.append(default_obs, nondefault_obs)
    
    model = LogisticRegression()
    
    # whether we have multiple independent variables or just one
    # and train  the  model
    if X.shape[1]== 1:
        model.fit(X[all_index], y[all_index])       
    else:
        model.fit(X.loc[all_index].reset_index(drop=True), y[all_index])
        
    return model

In [47]:
# train a logit model on the balanced data
# using overall_prediction index to predict default
balanced_model = train_balanced(overall_prediction.reshape(-1, 1), df_1c["default-next"])
print(balanced_model.coef_, balanced_model.intercept_)

[[0.01448595]] [-4.17165602]


In [54]:
# prepare a second dataset, validate
df_100 = read_sims_result("/Users/xcheng/Documents/Oberlin/Summer2/DataAnalysis/data/0719/1IR100s", 32)
mx_100n = cleanup_network(df_100, numNode=32, numPeriod=15, numSim=100)
df_100c = cleanup_0IR_exp(df_100, numNode=32, numPeriod=15, numSim=100)

edge_weights_100 = create_edge_weight(mx_100n, df_100, 0.6)
node_weights_100 = create_node_weight(mx_100n, df_100c, final, independent)

overall_pred_100 = customized_random_walk_exp(edge_weights_100, node_weights_100, random_walk_single_1b, 500)

accuracy = balanced_model.score(overall_pred_100.reshape(-1, 1), df_100c["default-next"])
conf = confusion_matrix(df_100c["default-next"], balanced_model.predict(overall_pred_100.reshape(-1, 1)))
tn, fp, fn, tp = conf.ravel()
print("{}\n accuracy:{:24}\n precision:{:24}\n recall:{:24}\n".format(
        conf, 
        accuracy, 
        tp/(tp+fp),
        tp/(tp+fn)))

[[33191  2885]
 [   20   310]]
 accuracy:      0.9202054606383563
 precision:     0.09702660406885759
 recall:      0.9393939393939394



In [192]:
# prediction based on logistic regression with balanced sheet information
bs_prediction_all = node_weights_all.reshape(1,-1)[0]
bal_sh_prediction = bs_prediction_all [bs_prediction_all != -1]

In [193]:
# zip two predictions into a dataframe
two_predictions = pd.DataFrame({'balanced sheet':bal_sh_prediction, 'network':overall_prediction})

In [196]:
# 222222222222222222222222222
# K-fold cross validation
# use approx. default probability + balanced sheet default prob to predict default
# This does not seem to be an effective approach
# ###########################
customized_k_fold_balanced(two_predictions, df_1c["default-next"])

[[0.82227979 0.01355207]] [-4.20212574]
[[1.32219723 0.01149829]] [-3.95625961]
[[1.39640341 0.01128357]] [-3.95008216]
[[0.93016178 0.01308366]] [-4.172605]
[[1.25382444 0.01195987]] [-4.0325485]
[[1.31984862 0.01175224]] [-3.9977899]
[[1.37974704 0.01172133]] [-4.00892144]
[[1.21228164 0.01169776]] [-3.94069053]
[[1.17756964 0.01191752]] [-3.98700097]
[[1.12306241 0.01205025]] [-3.90697603]
[[1.21085247 0.01178642]] [-4.02767211]
[[1.36758064 0.01173838]] [-4.02730538]
[[148  13]
 [  5 156]]
 accuracy:       0.944207027540361
 precision:      0.9230769230769231
 recall:       0.968944099378882



Using network information (4 modified page rank I created) as the sole predictor seems to predict the result fairly well. It is significantly better than using a single balanced sheet property or balanced sheet default prediction (logisitis regression) to do the same thing.

Combining (logsitic regression) balanced sheet default prediction (logisitis regression) with network information (4 modified page rank I created) does not improve results significantly.

----------------------------------------------------------------------
Stuff Below this are old stuff that I might or might not need.
----------------------------------------------------------------------
----------------------------------------------------------------------

In [125]:
def dist_avg_max(N):
    """
    calculate average & max distances between all pair of nodes
    
    Parameters
    ----------
    N: 4D numpy array [n_simulations, n_periods, n_borrowers, n_lenders]
        debt adjacency matrices 
    
    Returns
    ----------
    avg_d: 2D numpy array [n_simulations, n_periods]
        average distances between all pair of nodes
    max_d: 2D numpy array [n_simulations, n_periods]
        max distances between all pair of nodes
    """
    numSim, numPeriod, _, _ = N.shape
    avg_d = np.empty((numSim, numPeriod-2))
    max_d = np.empty((numSim, numPeriod-2))
    
    for s in range(numSim):
        for p in range(1,numPeriod-1):
            disG = nx.DiGraph(N[s,p])
            dists = shortest_path_length(disG, weight=None)
            curlist=[]
            for source in dists:
                curlist.extend(source[1].values())
            avg_d[s,p-1] = sum(curlist) / float(len(curlist))
            max_d[s,p-1] = max(curlist)
            
    return avg_d, max_d

In [120]:
# ###########################
# Visualize max/avg distances between banks
# ###########################
# avgg, maxx = dist_avg_max(mx_1n)
#
# pavg = pd.DataFrame(avgg)
# pmax = pd.DataFrame(maxx)
# # pavg.mean().plot()
# abc = pmax.stack().value_counts().sort_index().plot(
#     kind="bar",
#     title="max distances, 1 interest rates, 50 simulations, 15 periods",
#     figsize=(8,6),
#     fontsize=12
# )
# abc.set_xlabel("max distance between any pair of reachable nodes")
# abc.set_ylabel("frequncy")
# abc.title.set_fontsize(15)
# abc.xaxis.label.set_fontsize(15)
# abc.yaxis.label.set_fontsize(15)

In [6]:
def weigh_networks(N, model, variables):
    """
    Add weight to network
    Each debt is multiplied by lenders' predicted default probability
    
    Parameters
    ----------
    N: 4D numpy array [n_simulations, n_periods, n_borrowers, n_lenders]
        debt adjacency matrices 
    model: scikit learn LogisticRegression
        model for default probability
    variables: a list of strings
        independent variables for the model
    
    Returns
    ----------
    WN: 4D numpy array [n_simulations, n_periods, n_borrowers, n_lenders]
        new weighted debt adjacency matrices
    """
    
    WN = np.copy(N)
    simNum, periodNum, bankNum, _= N.shape
    
    for s in range(simNum):
        for p in range(1,periodNum-1):
            for b in range(bankNum):
                X = df_1c[np.array(df_1c["sim#"]==s) &
                          np.array(df_1c["period"]==p) & 
                          np.array(df_1c["bankID"]==b)
                         ][variables].values
                if X.any():
                    predicted_default_probability = model.predict_proba(X)[0][1]
                    WN[s, p-1, b] *= predicted_default_probability
                    
    return WN

In [46]:
def my_pagerank_numpy(G, alpha=0.85, personalization=None, weight='weight', dangling=None):
    """
    This is basically pagerank_numpy without normalization.
    """
    from networkx.algorithms.link_analysis.pagerank_alg import google_matrix
    
    if len(G) == 0:
        return {}
    M = google_matrix(G, alpha, personalization=personalization,
                      weight=weight, dangling=dangling)
    # use numpy LAPACK solver
    eigenvalues, eigenvectors = np.linalg.eig(M.T)
    ind = np.argmax(eigenvalues)
    # eigenvector of largest eigenvalue is at ind
    largest = np.array(eigenvectors[:, ind]).flatten().real
    return dict(zip(G, map(float, abs(largest))))

In [8]:
def apply_to_networks(f, N):
    """
    Calculate Page Rank scores for all the networks 
    
    Parameters
    ----------
    f: function (2D numpy array -> matrix)
        the function to apply to each network (e.g. Page Rank)
    N: 4D numpy array [n_simulations, n_periods, n_borrowers, n_lenders]
        debt adjacency matrices (netowrks)
    
    Returns
    ----------
    PG: 3D numpy array [n_simulations, n_periods, n_banks]
        Page Rank scores
    """
    
    simNum, periodNum, bankNum, _= N.shape
    PG = np.empty((simNum, periodNum, bankNum))
    
    for s in range(simNum):
        for p in range(1,periodNum-1):
            PG[s, p] = np.array(list(f(nx.DiGraph(N[s, p])).values()))
            
    return PG

In [9]:
# ###########################
# Let's add the weight
# ###########################
weighted = weigh_networks(mx_1n, final, independent)

In [50]:
# ###########################
# Let's calculate pagerank
# ###########################
pg_iter = apply_to_networks(pagerank, weighted)
pg_norm = apply_to_networks(pagerank_numpy, weighted)
pg_not_norm = apply_to_networks(my_pagerank_numpy, weighted)